# 35 장. 객체의 사용 관계 이해하기

## 35.1. 사용 관계(Association)

In [ ]:
class Master:
    def __init__(self, name):
        self._name = name
        self.Name = Slave()
        
class Slave:
    def get_name(self, other):
        return other._name

In [ ]:
m = Master('고요한')

In [ ]:
m.Name.get_name(m)

## 35.2. 결합 관계(Aggregation)

In [ ]:
class Balance(object):
    def __init__(self, account_code, money=0):
        self.account_code = account_code
        self.balance = money
        
    def set_balance(self, money):
        self.balance += money

In [ ]:
class Agreement(object):
    def __init__(self, money, b):
        self.acc_balance = money
        self.b = b
        self.b.set_balance(money)
        
    def __del__(self):
        self.b.set_balance(-self.acc_balance)
        del self

In [ ]:
b = Balance('1234')

In [ ]:
a = Agreement(100, b)

In [ ]:
a.__dict__

In [ ]:
b.__dict__

In [ ]:
del a

In [ ]:
b

In [ ]:
b.__dict__

## 35.3. 조합 관계(Composition)

In [ ]:
class Balance(object):
    def __init__(self, money=0):
        self._bal = 0
        self.set_balance(money)
        
    def set_balance(self, money):
        self._bal += money

In [ ]:
class Agreement(object):
    def __init__(self, acc_no, money):
        self.acc_no = acc_no
        self.balance = Balance(money)
        
    def get_balance(self):
        return self.balance._bal

In [ ]:
a = Agreement('1234', 1000)

In [ ]:
a.__dict__

In [ ]:
a.get_balance()

In [ ]:
del a

In [ ]:
a.get_balance()

## 35.4. 의존 관계(Dependency)

In [ ]:
class Balance(object):
    
    def __init__(self, money=0):
        self._bal = 0
        
    def get_balance(self):
        return self._bal
    
    def set_balance(self, acc_no, money):
        if not hasattr(self, '_acc_no'):
            self._acc_no = acc_no
        self._bal += money
        
    def __del__(self):
        del self

In [ ]:
class Agreement(object):
    
    def __init__(self, acc_no, b, money):
        self.acc_no = acc_no
        self.set_balance(b, money)
        
    def get_balance(self, b):
        return b.get_balance()
    
    def set_balance(self, b, money):
        b.set_balance(self.acc_no, money)
        
    def __del__(self):
        del self

In [ ]:
b = Balance()

In [ ]:
a = Agreement('1234', b, 1000)

In [ ]:
a.__dict__

In [ ]:
b.__dict__

In [ ]:
a.get_balance(b)

In [ ]:
del a

In [ ]:
a.get_balance(b)

In [ ]:
b.__dict__

In [ ]:
del b

In [ ]:
b

## 35.5. 옵저버(Observer) 패턴 알아보기

In [ ]:
from abc import ABC, abstractmethod

In [ ]:
from random import randrange

In [ ]:
class StateMachine(ABC):
    
    @abstractmethod
    def attach(self, observer):
        pass
    
    @abstractmethod
    def detach(self, observer):
        pass
    
    @abstractmethod
    def notify(self):
        pass

In [ ]:
class ConcreteStateMachine(StateMachine):
    
    _state = 0
    _observers = []
    
    def attach(self, observer):
        print('옵저버 등록')
        self._observers.append(observer)
        
    def detach(self, observer):
        print('옵저버 삭제')
        self._observers.remove(observer)
        
    def notify(self):
        print('등록된 옵저버에 통보...')
        for observer in self._observers:
            observer.update(self)
            
    def some_business_logic(self):
        print('상태 변경하기')
        self._state = randrange(0, 10)
        
        print(f'상태 값 확인하기: {self._state}')
        self.notify()

In [ ]:
class Observer(ABC):
    
    @abstractmethod
    def update(self, subject):
        pass

In [ ]:
class ConcreteObserverA(Observer):
    def update(self, subject):
        if subject._state <= 9:
            print('관찰 상태 A: 이벤트 실행')
            
class ConcreteObserverB(Observer):
    def update(self, subject):
        if subject._state == 0 or subject._state >=2:
            print('관찰 상태 B: 이벤트 실행')            

In [ ]:
state = ConcreteStateMachine()

In [ ]:
observer_a = ConcreteObserverA()
state.attach(observer_a)

In [ ]:
observer_b = ConcreteObserverB()
state.attach(observer_b)

In [ ]:
state.some_business_logic()

In [ ]:
state.detach(observer_a)

In [ ]:
state.some_business_logic()